In [5]:
# %pip install --upgrade langchain langsmith langgraph langchain_openai

In [6]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OPENAI_API_KEY from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Agent Demo"

### Set up the Tools

In [7]:
from weaviate import Client

client = Client("http://localhost:8080")


def query_collection(query):
    response = (
        client.query.get("code_example", ["code"])
        .with_near_text({"concepts": [query]})
        .with_limit(3)
        .do()
    )

    formatted_response = ""
    for result in response["data"]["Get"]["Code_example"]:
        formatted_response += "# Example:\n"
        formatted_response += f"{result['code']}\n\n\n"

    return formatted_response

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda

# Set up the system template with a variable for context
system_template = """
Generate working code for a Jupyter Notebook based on the user's request. Your code should use LangChain, and specifically use LangChain's Expression Language in structuring your code.

Strictly adhere to the code examples delimited by triple backticks below as context for how LangChain's API works. DO NOT use any patterns that you do not find in the example below, unless you are 100% certain they work in LangChain:

```
{context}
```

Before sharing, double check your work. I will tip you $100 if your code is perfect.

Do not explain your work, just share working code.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# Set up the human template with a variable for the request
human_template = """
{request}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
output_parser = StrOutputParser()

code_writing_runnable = prompt | model | output_parser

### Set up the Agent

In [9]:
### Set up the agent's tools

tools = []
pseudo_tools_visible = [
    "Retrieve Context",
    "Write Code",
    "Review Code",
    # "Save Code",
]
pseudo_tools_hidden = [
    "Store Request",
]

agent_tools = tools + pseudo_tools_visible + pseudo_tools_hidden

In [10]:
from langchain.pydantic_v1 import BaseModel
from enum import Enum
from langchain.tools import StructuredTool


# Set the agent options, which is FINISH plus all tools, with the exception of the hidden tools
agent_options = ["FINISH"] + agent_tools
agent_options = [item for item in agent_options if item not in pseudo_tools_hidden]

RouteOptions = Enum("RouteOptions", {option: option for option in agent_options})


class RouteInput(BaseModel):
    next: RouteOptions


def route(route: str) -> str:
    return route


router = StructuredTool.from_function(
    func=route,
    name="route",
    description="Select the next team member to use",
    args_schema=RouteInput,
    return_direct=False,
)

In [11]:
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)

system_prompt_initial = """
You are a supervisor tasked with managing a development team consisting of the following members: {members}.

Given the following feature request from a user, respond with the worker to act next.

Each worker will perform a task and respond with their results and status. This task is complete as soon as you know a worker has retrieved context related to the user's feature request. When the task is complete, respond with FINISH.

You typically follow this pattern:

1) Retrieve context related to the user's query. This is a REQUIRED step before writing code
2) Write code to solve the problem
3) Save the code you have written once the reviewer has approved the code
"""

# Get the prompt to use - you can modify this!
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_initial),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
).partial(options=str(agent_options), members=", ".join(agent_tools))

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, [router], prompt)

### Set up the Agent State

In [12]:
from typing import TypedDict, Annotated, Sequence, Union
import operator
from langchain_core.messages import BaseMessage
from langchain_core.agents import AgentAction, AgentFinish

class AgentState(TypedDict):
    # The list of previous messages in the conversation
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    # The user's original request
    original_request: str
    # The context for code writing
    context: str
    # The code sample being generated
    code: str
    # Track whether the code hsa been approved
    code_approved: bool

### Set up the node actions

In [13]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage, AIMessage


# Define the function that determines whether to continue or not
def should_continue(state):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(state["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"


# Define the function that calls the model
def call_model(state):
    # messages = state['messages']
    agent_outcome = agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}


def call_set_initial_state(state):
    messages = state["messages"]
    last_message = messages[-1]
    return {"original_request": last_message.content}


# Define the function to execute tools
def call_tool(state):
    # We construct an ToolInvocation from the function_call
    tool = state['agent_outcome'].tool_input['next']
    print("Running Tool: ", tool)

    if tool == "Retrieve Context":
        print("Retreive Context")
        context = query_collection(state["original_request"])
        new_message = AIMessage(content="You have context now")
        return {"context": context, "messages": [new_message]}
    elif tool == "Write Code":
        print("successfully writing code now")
        code_writing_runnable.invoke({"context": state["context"], "request": state["original_request"]})
        new_message = AIMessage(content="You have code now")
        return {"code": new_message, "messages": [new_message]}
    elif tool == "Review Code":
        print("successfully reviewing code now")
        new_message = AIMessage(content="Code is approved")
        return {"code_approved": True, "messages": [new_message]}
    elif tool == "Save Code":
        print("Save Code")
    return

### Define the Graph

In [14]:
from langgraph.graph import StateGraph, END

# Define a new graph
graph = StateGraph(AgentState)

# Define the two nodes we will cycle between
graph.add_node("agent", call_model)
graph.add_node("action", call_tool)
graph.add_node("initial_state", call_set_initial_state)

# Set the entrypoint
graph.set_entry_point("initial_state")

# Add a conditional edge
graph.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

# Aadd the Normal Edges
graph.add_edge("action", "agent")
graph.add_edge("initial_state", "agent")

# Compile it
app = graph.compile()

### Run our agent

In [15]:
from langchain_core.messages import HumanMessage

feature_request = """
Create a chain that does the following:
- Accept a string named answer as input
- Format a System and Human message using templates. The System message has output instructions via Pydantic. The Human message uses answer as context. Output instructions should require format to a Pydantic schema for hmw_question with a question (up to 10 words) and a role (either CMO, CTO, or CEO) 
- Pass the messages to OpenAI
- Parse the response using Pydantic
"""

inputs = {"messages": [HumanMessage(content=feature_request)]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'initial_state':
---
{'original_request': '\nCreate a chain that does the following:\n- Accept a string named answer as input\n- Format a System and Human message using templates. The System message has output instructions via Pydantic. The Human message uses answer as context. Output instructions should require format to a Pydantic schema for hmw_question with a question (up to 10 words) and a role (either CMO, CTO, or CEO) \n- Pass the messages to OpenAI\n- Parse the response using Pydantic\n'}

---

Output from node 'agent':
---
{'agent_outcome': AgentActionMessageLog(tool='route', tool_input={'next': 'Retrieve Context'}, log="\nInvoking: `route` with `{'next': 'Retrieve Context'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"next":"Retrieve Context"}', 'name': 'route'}})])}

---

Running Tool:  Retrieve Context
Retreive Context
Output from node 'action':
---
{'context': '# Example:\n# Create a chain that does the f